In [6]:
%load_ext autoreload
%autoreload 2
from thermo_idm.cluster import Cluster

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'thermo_idm'